In [7]:
!pip install dice_ml

  Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.8 MB/s eta 0:00:0000:0100:01
Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import dice_ml
from dice_ml import Dice

In [17]:
# Step 1: Load and prepare the dataset
df = pd.read_csv("insurance.csv")

# Step 2: Define features and target
features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
target = 'charges'

X = df[features]
y = df[target]

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Preprocessing pipeline
cat_features = ['sex', 'smoker', 'region']
num_features = ['age', 'bmi', 'children']
target = 'charges'

preprocessor = ColumnTransformer([
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='Missing', add_indicator=True)),
        ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ]), cat_features),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
        ('scaler', StandardScaler())
    ]), num_features)
])

# Step 4: Define and train model
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42))
])
model_pipeline.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex', 'smoker', 'region']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy=...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [18]:
# Step 1: DiCE Setup
X_train_transformed = pd.DataFrame(
    preprocessor.transform(X_train),
    columns=preprocessor.get_feature_names_out()
)
X_train_transformed['charges'] = y_train.reset_index(drop=True)

data_dice = dice_ml.Data(
    dataframe=X_train_transformed,
    continuous_features=[col for col in X_train_transformed.columns if col != 'charges'],
    outcome_name='charges'
)

model_dice = dice_ml.Model(
    model=model_pipeline.named_steps['regressor'],
    backend='sklearn',
    model_type='regressor'
)

# Step 2: Generate Counterfactuals
def generate_counterfactual(query_instance):
    query_transformed = preprocessor.transform(query_instance)
    query_df = pd.DataFrame(query_transformed, columns=preprocessor.get_feature_names_out())

    explainer = Dice(data_dice, model_dice, method='random')
    dice_exp = explainer.generate_counterfactuals(
        query_df,
        total_CFs=10,  # generate more and filter down
        desired_range=[0, 10000],
        features_to_vary='all'
    )

    return dice_exp.cf_examples_list[0].final_cfs_df

# Step 3: Inverse Transform and Filter
def inverse_transform_counterfactuals(counterfactuals_df, query_instance):
    cat_transformer = preprocessor.named_transformers_['cat']
    num_transformer = preprocessor.named_transformers_['num']

    ohe = cat_transformer.named_steps['encoder']
    cat_columns = ohe.get_feature_names_out(cat_features)
    cat_columns_prefixed = [f'cat__{col}' for col in cat_columns]
    num_columns_prefixed = [f'num__{col}' for col in num_features]

    encoded_cat_df = counterfactuals_df[cat_columns_prefixed]
    scaled_num_df = counterfactuals_df[num_columns_prefixed]

    recovered_cats = ohe.inverse_transform(encoded_cat_df)
    recovered_cat_df = pd.DataFrame(recovered_cats, columns=cat_features)

    num_transformer = preprocessor.named_transformers_['num']
    scaler = num_transformer.named_steps['scaler']
    imputer = num_transformer.named_steps['imputer']
    unscaled_num = scaler.inverse_transform(scaled_num_df)
    unscaled_num = imputer.inverse_transform(unscaled_num)
    recovered_num_df = pd.DataFrame(unscaled_num, columns=num_features)

    # Round children to integer
    if 'children' in recovered_num_df.columns:
        recovered_num_df['children'] = recovered_num_df['children'].round().astype(int)

    readable_df = pd.concat([recovered_cat_df, recovered_num_df], axis=1)

    # Add predicted charges
    if 'charges' in counterfactuals_df.columns:
        readable_df['charges'] = counterfactuals_df['charges'].values

    # Drop duplicates *based only on features*, ignore charges
    feature_columns = [col for col in readable_df.columns if col != 'charges']
    readable_df = readable_df.drop_duplicates(subset=feature_columns)

    # Filter out exact duplicates of original input
    differences = readable_df[features].ne(query_instance.iloc[0]).any(axis=1)
    filtered_df = readable_df[differences].reset_index(drop=True)

    return filtered_df

# Step 4: Usage and Save
if __name__ == "__main__":
    query_instance = pd.DataFrame([{
        'age': 35,
        'sex': 'female',
        'bmi': 33.0,
        'children': 2,
        'smoker': 'yes',
        'region': 'southeast'
    }])

    original_pred = model_pipeline.predict(query_instance)[0]
    print("\nOriginal Input with Predicted Charge:")
    print(query_instance.assign(predicted_charge=np.round(original_pred, 2)))

    cf_df = generate_counterfactual(query_instance)
    readable_cf_df = inverse_transform_counterfactuals(cf_df, query_instance)

    print("\nReadable & Unique Counterfactual Explanations:")
    print(readable_cf_df)

    # Save to file
    readable_cf_df.to_csv("counterfactuals_readable.csv", index=False)
    print("✅ Counterfactuals saved to 'counterfactuals_readable.csv'")



Original Input with Predicted Charge:
   age     sex   bmi  children smoker     region  predicted_charge
0   35  female  33.0         2    yes  southeast      40052.820312


100%|██████████| 1/1 [00:00<00:00, 12.42it/s]


Readable & Unique Counterfactual Explanations:
      sex smoker     region        age   bmi  children  charges
0  female     no  southeast  38.215880  33.0         2   7593.0
1  female     no  northwest  35.000000  33.0         2   7517.0
2  female     no  southeast  35.000000  33.0         2   8045.0
3  female     no  southeast  35.119668  33.0         2   7517.0
4  female     no  southeast  18.417842  33.0         2   5386.0
✅ Counterfactuals saved to 'counterfactuals_readable.csv'
